<a href="https://colab.research.google.com/github/hiroporn/moji-okoshi-srt/blob/main/moji_okoshi_srt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title GoogleDriveをマウントする

from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#@title データを入れるフォルダ作成

import os

# 環境にフォルダを２つ作る

checkContentFolder = os.path.exists("input")
checkDownLoadFolder = os.path.exists("output")
if not checkContentFolder:
    os.mkdir("input")
if not checkDownLoadFolder:
    os.mkdir("output")

# 最初にマウントしたGoogleドライブから音声データをinputに入れる(D&Dも可能)

In [ ]:
#@title Whisperとsrtの実装

!pip install srt
!pip install git+https://github.com/openai/whisper.git

In [ ]:
#@title 実際に文字起こしを実行する

# 文字起こしされたものは、srtファイルで出力する

import srt
import whisper

from srt import Subtitle
from datetime import timedelta

# ファイル名は右側で入れられるように小細工

file_name = "ongen.mp3"#@param {type:"string"}

# モデルをsmallにするとCPUの場合で、ほぼ等倍程度で出力されるっぽい？
# GPUが使える場合でも、時間的なものを考えると、mediumがバランスがよさそう。
# それと、largeのほうが精度は良いが、時間もかかる。

model = whisper.load_model("medium")

# 録音データに無音時間が無い場合は、tenperatureの設定0.8とか。min=0.7をしても良いかも？
# 現状で設定していないのは、長めの暫時休憩があったデータで、同じ単語を繰り返して崩壊したため
# result = model.transcribe(f"content/{file_name}", language="ja",  temperature=0.8)
# without_timestamps=Trueにすると、30秒区切りになるが、内容をロストするようだ。

result = model.transcribe(f"input/{file_name}",
                          temperature=0.8,
                          language="ja",
                          verbose=False,
                          without_timestamps=False,
                          )

# 文字を50文字程度で折返す設定。どっかからのコピペしたもの

def add_line(s):
    new_s = s
    s_count = len(s)
    s_max_count = 60
    if s_count >= s_max_count:
        if (s_count - s_max_count) >= 3:
            # 60文字以上、かつ、2行目が3文字以上あれば、改行する。
            # つまり、63文字以上であれば、60文字で改行することになる。
            new_s = s[:s_max_count] + "\n" + s[s_max_count:]
 
    return new_s

segments = result["segments"]

subs = []

# 文字起こしした生データから、必要なところを抜き出すループ。

for data in segments:
    index = data["id"] + 1
    start = data["start"]
    end = data["end"]
    text = add_line(data["text"])
    sub = Subtitle(index=1, start=timedelta(seconds=timedelta(seconds=start).seconds, microseconds=timedelta(seconds=start).microseconds), 
                   end=timedelta(seconds=timedelta(seconds=end).seconds, microseconds=timedelta(seconds=end).microseconds), content=text, proprietary='')
    subs.append(sub)
 
# 出力フォルダ(output)にsrtファイルを書き出す

with open(f'output/{file_name}' + ".srt", mode="w", encoding="utf-8") as f:
    f.write(srt.compose(subs))

# できあがりの表示をする。

print('完了！')
